In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# φ-РЕШЁТКА — ПОЛНЫЙ ТЕСТЕР ГИПОТЕЗ (11D→13D) ДЛЯ COLAB
# Электрон/Нейтрон фокус • Без файлов • Много графиков/виджетов
# ============================================================

import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from math import log
from textwrap import dedent

# ----------- (опционально) интерактив ----------
USE_WIDGETS = True
try:
    import ipywidgets as W
    from IPython.display import display, clear_output
except Exception:
    USE_WIDGETS = False

# ------------------ КОНСТАНТЫ -------------------
PHI = (1 + 5**0.5) / 2.0
ALPHA = PHI
BASE = PHI**ALPHA            # φ^φ
ME = 0.510999                # масса электрона, МэВ (нормировочный якорь)

# ------------------ ДАННЫЕ ----------------------
# Формат строк: (name, family, mass_MeV, n, k, c, spin, charge, strangeness)
PARTICLES_DATA = [
    # Лептоны
    ("electron", "lepton", 0.510999, 0, 0, 1, 0.5, -1, 0),
    ("muon", "lepton", 105.658000, 1, 6, 3, 0.5, -1, 0),
    ("tau", "lepton", 1776.860000, 2, 9, 4, 0.5, -1, 0),

    # Кварки
    ("up", "quark", 2.160000, -7, 8, 5, 0.5, 2/3, 0),
    ("down", "quark", 4.670000, -6, 8, 5, 0.5, -1/3, 0),
    ("strange", "quark", 93.400000,  1, 5, 3, 0.5, -1/3, -1),
    ("charm", "quark",   1270.0,     2, 8, 3, 0.5,  2/3,  0),
    ("bottom", "quark",  4180.0,     3, 9, 3, 0.5, -1/3,  0),
    ("top", "quark",   172760.0,     7,15, 3, 0.5,  2/3,  0),

    # Барионы
    ("proton",  "baryon", 938.272081, 2, 8, 2, 0.5,  1, 0),
    ("neutron", "baryon", 939.565413, 2, 8, 2, 0.5,  0, 0),

    # Мезоны
    ("pi0",         "meson", 134.976800, 1, 6, 2, 0, 0, 0),
    ("pi_charged",  "meson", 139.570390, 1, 6, 2, 0, 1, 0),
    ("K_charged",   "meson", 493.677000, 2, 7, 2, 0, 1, 1),
    ("K0",          "meson", 497.611000, 2, 7, 2, 0, 0, 1),
    ("eta",         "meson", 547.862000, 2, 7, 2, 0, 0, 0),
    ("eta_prime",   "meson", 957.780000, 2, 8, 2, 0, 0, 0),
    ("Jpsi",        "meson", 3096.900000, 3, 9, 2, 1, 0, 0),
    ("Upsilon",     "meson", 9460.300000, 4,10, 2, 1, 0, 0),
    ("rho770",      "meson", 775.260000,  2, 8, 2, 1, 1, 0),
    ("omega782",    "meson", 782.650000,  2, 8, 2, 1, 0, 0),
    ("phi1020",     "meson", 1019.460000, 2, 8, 2, 1, 0, 1),
    ("D0",          "meson", 1864.840000, 2, 9, 2, 0, 0, 0),
    ("D_charged",   "meson", 1869.660000, 2, 9, 2, 0, 1, 0),
    ("B0",          "meson", 5279.650000, 3,10, 2, 0, 0, 0),
    ("B_charged",   "meson", 5279.340000, 3,10, 2, 0, 1, 0),

    # Бозоны
    ("W",     "boson", 80379.0,  5,13, 2, 1, 1, 0),
    ("Z",     "boson", 91188.0,  5,13, 2, 1, 0, 0),
    ("Higgs", "boson",125090.0,  5,13, 3, 0, 0, 0),
]

COLS = ['name','family','mass_MeV','n','k','c','spin','charge','strangeness']
df0 = pd.DataFrame(PARTICLES_DATA, columns=COLS)

# ---- (опционально) приблизительные τ и магн. моменты (можно отключить) ----
USE_PDG_APPROX = True  # переключатель: вкл/выкл встроенные приближения

tau_approx = {
    # секунд (≈, порядок)
    "electron": float('inf'),       # стабильный
    "muon": 2.196981e-6,
    "tau": 2.903e-13,

    "pi0": 8.52e-17,
    "pi_charged": 2.6033e-8,
    "K_charged": 1.238e-8,
    "K0": 5.116e-8,  # K_L (пример)
    "eta": 5.0e-19,  # порядок
    "eta_prime": 3.3e-21,  # порядок
    "Jpsi": 7.2e-21,
    "Upsilon": 1.2e-20,
    "rho770": 4.5e-24,
    "omega782": 7.7e-23,
    "phi1020": 1.55e-22,
    "D0": 4.1e-13,
    "D_charged": 1.04e-12,
    "B0": 1.52e-12,
    "B_charged": 1.64e-12,
    "W": 3e-25,
    "Z": 3e-25,
    "Higgs": 1.6e-22,  # порядок
    "proton": float('inf'),
    "neutron": 880.2,  # сек
}

# магнитные моменты (в ядерных магнетонах μ_N для барионов; для e — в μ_B с пометкой)
magmom_approx = {
    "proton": +2.792847,  # μ_N
    "neutron": -1.913043, # μ_N
    "electron_muB": -1.00115965218, # аномалия g/2-1 в μ_B (здесь справочный маркер)
}

# ------------------ УТИЛИТЫ ----------------------
def mass_pred(n, k):
    return ME * (2.0**n) * (BASE**k)

def log_phi(x):
    return math.log(x) / math.log(PHI)

def safe_get(series, name, default=np.nan):
    try:
        return series.loc[name]
    except Exception:
        return default

def build_df(chi_coeffs=(2,1,1,0), delta_coeffs=(1,1,2,0), tau0=1.0, use_pdg=USE_PDG_APPROX):
    """
    chi_coeffs = (a,b,c,d) для χ11 = (a*n + b*k + c*c + d*charge) % 11
    delta_coeffs = (p,q,r,s) для δ13 = (p*n + q*k + r*c + s*charge) % 13
    tau0 — базовый масштаб для λ12 = round(log_phi(τ/tau0))
    """
    a,b,c,d = chi_coeffs
    p,q,r,s = delta_coeffs

    df = df0.copy()
    df['K'] = df['n'].abs() + df['k'].abs()
    df['k_mod6'] = df['k'] % 6
    df['mass_pred_MeV'] = df.apply(lambda r: mass_pred(r['n'], r['k']), axis=1)
    df['rel_err_%'] = (df['mass_pred_MeV'] - df['mass_MeV']).abs() / df['mass_MeV'] * 100.0

    # χ11, δ13
    df['chi11'] = (a*df['n'] + b*df['k'] + c*df['c'] + d*np.sign(df['charge'])) % 11
    df['delta13'] = (p*df['n'] + q*df['k'] + r*df['c'] + s*np.sign(df['charge'])) % 13

    # lifetimes
    if use_pdg:
        df['tau_s'] = df['name'].map(tau_approx).astype(float)
    else:
        df['tau_s'] = np.nan

    # λ12
    df['lambda12'] = np.where(np.isfinite(df['tau_s']),
                              np.round(np.log(df['tau_s']/tau0)/np.log(PHI)).astype('float'),
                              np.nan)

    # магн. моменты для p,n (μ_N); для e — отдельный маркер
    df['mu_N'] = np.nan
    df.loc[df['name']=='proton','mu_N']  = safe_get(pd.Series(magmom_approx), 'proton', np.nan)
    df.loc[df['name']=='neutron','mu_N'] = safe_get(pd.Series(magmom_approx), 'neutron', np.nan)
    df['muB_e'] = np.nan
    df.loc[df['name']=='electron','muB_e'] = safe_get(pd.Series(magmom_approx), 'electron_muB', np.nan)

    return df

def occupied_empty_nodes(df, n_min=-8, n_max=8, k_min=0, k_max=16):
    occupied = set(zip(df['n'], df['k']))
    all_nodes = [(n,k) for n in range(n_min, n_max+1) for k in range(k_min, k_max+1)]
    empty = [(n,k) for (n,k) in all_nodes if (n,k) not in occupied]
    return occupied, empty, all_nodes

def candidates_by_delta(df, empty, c_guess=2, match='neutron', top=30):
    d_target = int(df.loc[df['name']==match,'delta13'].iloc[0])
    rows = []
    for (n,k) in empty:
        K = abs(n)+abs(k)
        d = (n + k + 2*c_guess) % 13  # default form used в поиске кандидатов
        if d == d_target:
            mp = mass_pred(n,k)
            rows.append((K,n,k,d,mp))
    cdf = pd.DataFrame(rows, columns=['K','n','k','delta13','mass_pred_MeV']).sort_values(['K','mass_pred_MeV']).head(top)
    return cdf

# ------------------ ОТЧЁТ ------------------------
def print_hyp_tests(df):
    p = df[df['name']=='proton'].iloc[0]
    n = df[df['name']=='neutron'].iloc[0]

    print("="*80)
    print("H11 — χ (mod 11)")
    print("="*80)
    print("\nH11.1 k-паритет по семействам:")
    pt = df.pivot_table(index='family', columns='k_parity', values='name', aggfunc='count').fillna(0).astype(int) \
         if 'k_parity' in df.columns else None
    if pt is None:
        tmp = df.copy()
        tmp['k_parity'] = df['k'] % 2
        pt = tmp.pivot_table(index='family', columns='k_parity', values='name', aggfunc='count').fillna(0).astype(int)
    print(pt)

    print("\nH11.2 χ-индикатор лептонов:")
    print(df[df['family']=='lepton'][['name','n','k','c','chi11']])

    delta_chi11 = (int(n['chi11']) - int(p['chi11'])) % 11
    print("\nH11.3 Δχ11(neutron, proton) (твоя проверка):")
    print(f"  χ11(p)={int(p['chi11'])}, χ11(n)={int(n['chi11'])}, Δ={delta_chi11} (mod 11)")

    print("\n" + "="*80)
    print("H12 — λ (лог-φ шкала времени жизни)")
    print("="*80)
    if df['tau_s'].notna().any():
        print(df[['name','family','tau_s','lambda12']].dropna().sort_values('lambda12').to_string(index=False))
        if np.isfinite(p['tau_s']) and np.isfinite(n['tau_s']):
            print(f"\n τ(p) = {'∞' if math.isinf(p['tau_s']) else p['tau_s']} s, τ(n) = {n['tau_s']} s")
    else:
        print("Нет данных по τ: выставь USE_PDG_APPROX=True или заполни свои lifetimes.")

    print("\n" + "="*80)
    print("H13 — δ (mod 13)")
    print("="*80)
    print("\nH13.1 класс W/Z/H:")
    print(df[df['family']=='boson'][['name','n','k','c','delta13']].to_string(index=False))

    delta_np = (int(n['delta13']) - int(p['delta13'])) % 13
    print("\nH13.2 Δδ(neutron, proton) (твоя проверка):")
    print(f"  δ(p)={int(p['delta13'])}, δ(n)={int(n['delta13'])}, Δ={delta_np} (mod 13)")

def make_plots(df, cand_df=None, show=True):
    families = df['family'].unique().tolist()
    fam_colors = {
        'lepton':'#FF6B6B', 'quark':'#4ECDC4', 'baryon':'#45B7D1',
        'meson':'#FFA07A', 'boson':'#98D8C8'
    }

    plt.rcParams['figure.figsize'] = (22, 16)
    fig = plt.figure(constrained_layout=True)
    gs = fig.add_gridspec(3, 3)

    # 1) n–k карта
    ax1 = fig.add_subplot(gs[0,0])
    for f in families:
        dff = df[df['family']==f]
        ax1.scatter(dff['n'], dff['k'], s=180, alpha=0.75, label=f,
                    edgecolors='black', linewidth=1.3, c=fam_colors.get(f,'#999999'))
        for _,r in dff.iterrows():
            ax1.annotate(r['name'][:4], (r['n'],r['k']), ha='center', va='bottom', fontsize=8)
    ax1.axhline(0, color='black', lw=0.5); ax1.axvline(0, color='black', lw=0.5)
    ax1.grid(True, alpha=0.3); ax1.set_title('Карта различимости (n,k)')
    ax1.set_xlabel('n'); ax1.set_ylabel('k')
    ax1.legend(fontsize=9)

    # 2) Теплокарта занятости
    ax2 = fig.add_subplot(gs[0,1])
    n_min, n_max = df['n'].min()-1, df['n'].max()+1
    k_min, k_max = max(0, df['k'].min()-1), df['k'].max()+1
    n_range = list(range(n_min, n_max+1))
    k_range = list(range(k_min, k_max+1))
    grid = np.zeros((len(k_range), len(n_range)))
    for _,r in df.iterrows():
        i = n_range.index(r['n']); j = k_range.index(r['k'])
        grid[j, i] += 1
    im = ax2.imshow(grid, aspect='auto', origin='lower',
                    extent=[n_min-0.5,n_max+0.5,k_min-0.5,k_max+0.5], cmap='YlOrRd')
    fig.colorbar(im, ax=ax2, fraction=0.046, pad=0.04)
    ax2.set_title('Тепловая карта занятости узлов')
    ax2.set_xlabel('n'); ax2.set_ylabel('k'); ax2.grid(True, alpha=0.15, color='white')

    # 3) K vs log10(mass)
    ax3 = fig.add_subplot(gs[0,2])
    for f in families:
        dff = df[df['family']==f]
        ax3.scatter(dff['K'], np.log10(dff['mass_MeV']), s=160, alpha=0.75,
                    edgecolors='black', linewidth=1.2, c=fam_colors.get(f,'#999'))
    ax3.set_title('Сложность K vs log10(масса)')
    ax3.set_xlabel('K'); ax3.set_ylabel('log10(M/MeV)')
    ax3.grid(True, alpha=0.3)

    # 4) k mod 6 по семействам (столбики)
    ax4 = fig.add_subplot(gs[1,0])
    width = 0.15; x = np.arange(6)
    for i,f in enumerate(families):
        counts = df[df['family']==f]['k'].mod(6).value_counts().reindex(range(6), fill_value=0).values
        ax4.bar(x + i*width, counts, width, alpha=0.75, edgecolor='black',
                color=fam_colors.get(f,'#999'), label=f)
    ax4.set_xticks(x + width*(len(families)-1)/2)
    ax4.set_xticklabels(range(6))
    ax4.set_title('Шестилучевая структура k mod 6')
    ax4.set_xlabel('k mod 6'); ax4.set_ylabel('кол-во')
    ax4.grid(True, axis='y', alpha=0.3); ax4.legend(fontsize=9)

    # 5) Распределение χ11 по семействам
    ax5 = fig.add_subplot(gs[1,1])
    uniq = sorted(df['chi11'].unique())
    x = np.arange(len(uniq)); width=0.12
    for i,f in enumerate(families):
        counts = df[df['family']==f]['chi11'].value_counts().reindex(uniq, fill_value=0).values
        ax5.bar(x + i*width, counts, width, alpha=0.8, edgecolor='black',
                color=fam_colors.get(f,'#999'), label=f)
    ax5.set_xticks(x + width*(len(families)-1)/2)
    ax5.set_xticklabels(uniq)
    ax5.set_title('Распределение χ11')
    ax5.set_xlabel('χ11'); ax5.set_ylabel('кол-во')
    ax5.grid(True, axis='y', alpha=0.3)

    # 6) Распределение δ13 по семействам
    ax6 = fig.add_subplot(gs[1,2])
    uniq = sorted(df['delta13'].unique())
    x = np.arange(len(uniq)); width=0.12
    for i,f in enumerate(families):
        counts = df[df['family']==f]['delta13'].value_counts().reindex(uniq, fill_value=0).values
        ax6.bar(x + i*width, counts, width, alpha=0.8, edgecolor='black',
                color=fam_colors.get(f,'#999'), label=f)
    ax6.set_xticks(x + width*(len(families)-1)/2)
    ax6.set_xticklabels(uniq)
    ax6.set_title('Распределение δ13')
    ax6.set_xlabel('δ13'); ax6.set_ylabel('кол-во')
    ax6.grid(True, axis='y', alpha=0.3)

    # 7) Кандидаты по δ=δ(neutron)
    ax7 = fig.add_subplot(gs[2,0])
    for f in families:
        dff = df[df['family']==f]
        ax7.scatter(dff['n'], dff['k'], s=140, alpha=0.6, label=f,
                    edgecolors='black', linewidth=1.0, c=fam_colors.get(f,'#999'))
    if cand_df is not None and len(cand_df):
        sc = ax7.scatter(cand_df['n'], cand_df['k'], s=(cand_df['K']*35).clip(60, 300),
                         marker='*', alpha=0.9, edgecolors='blue', linewidth=1.5, c='yellow', label='кандидаты (δ=δ_n)')
        for _,r in cand_df.head(10).iterrows():
            ax7.annotate(f"K={int(r['K'])}", (r['n'],r['k']), fontsize=8, ha='center', va='bottom')
    ax7.set_title('Кандидаты (совпадение δ с δ(neutron))')
    ax7.set_xlabel('n'); ax7.set_ylabel('k')
    ax7.axhline(0, color='black', lw=0.5); ax7.axvline(0, color='black', lw=0.5)
    ax7.grid(True, alpha=0.3); ax7.legend(fontsize=9)

    # 8) λ12 vs K (если есть τ)
    ax8 = fig.add_subplot(gs[2,1])
    if df['lambda12'].notna().any():
        for f in families:
            dff = df[(df['family']==f) & (df['lambda12'].notna())]
            if len(dff):
                ax8.scatter(dff['K'], dff['lambda12'], s=150, alpha=0.75,
                            edgecolors='black', linewidth=1.2, c=fam_colors.get(f,'#999'), label=f)
        # Подписи для neutron/proton/electron если есть λ
        for nm in ['electron','proton','neutron']:
            dff = df[df['name']==nm]
            if len(dff) and pd.notna(dff['lambda12'].iloc[0]):
                ax8.annotate(nm, (dff['K'].iloc[0], dff['lambda12'].iloc[0]),
                             fontsize=10, fontweight='bold')
        ax8.set_title('λ12 (лог-φ τ) vs K')
        ax8.set_xlabel('K'); ax8.set_ylabel('λ12')
        ax8.grid(True, alpha=0.3)
    else:
        ax8.axis('off'); ax8.text(0.5,0.5, "λ12 недоступно (нет τ)", ha='center', va='center')

    # 9) Ошибка φ-массы vs частица
    ax9 = fig.add_subplot(gs[2,2])
    dfx = df[['name','rel_err_%']].sort_values('rel_err_%')
    ax9.barh(dfx['name'], dfx['rel_err_%'], edgecolor='black')
    ax9.set_title('Ошибка |m_pred - m_obs| / m_obs · 100%')
    ax9.set_xlabel('проценты'); ax9.grid(True, axis='x', alpha=0.3)

    if show:
        plt.show()
    return fig

# ------------------ РАН ЗАПУСК -------------------
def run(chi_a=2, chi_b=1, chi_c=1, chi_d=0,
        del_p=1, del_q=1, del_r=2, del_s=0,
        tau0=1.0, use_pdg=USE_PDG_APPROX, c_guess=2,
        n_min=-8, n_max=8, k_min=0, k_max=16, show_plots=True):
    df = build_df(chi_coeffs=(chi_a,chi_b,chi_c,chi_d),
                  delta_coeffs=(del_p,del_q,del_r,del_s),
                  tau0=tau0, use_pdg=use_pdg)
    # добить k_parity (для таблицы) и печать
    df['k_parity'] = df['k'] % 2
    print_hyp_tests(df)

    # кандидаты
    occ, empty, all_nodes = occupied_empty_nodes(df, n_min, n_max, k_min, k_max)
    cand_df = candidates_by_delta(df, empty, c_guess=c_guess, match='neutron', top=30)
    print("\nH13.3 кандидаты (δ совпадает с нейтроном), топ-30:")
    if len(cand_df):
        print(cand_df.to_string(index=False))
    else:
        print("  — не найдено в заданной области.")

    # графика
    if show_plots:
        make_plots(df, cand_df, show=True)
    return df, cand_df

# ================== ИНТЕРАКТИВ ==================
if USE_WIDGETS:
    style = {'description_width':'120px'}
    layout = W.Layout(width='350px')
    ui = W.VBox([
        W.HTML("<h3>φ-решётка — интерактивный тест H11–H13</h3>"),
        W.HBox([
            W.VBox([
                W.HTML("<b>χ11 = (a·n + b·k + c·c + d·sign(charge)) mod 11</b>"),
                W.IntSlider(value=2, min=-9, max=9, step=1, description='a (n)', style=style, layout=layout),
                W.IntSlider(value=1, min=-9, max=9, step=1, description='b (k)', style=style, layout=layout),
                W.IntSlider(value=1, min=-9, max=9, step=1, description='c (c)', style=style, layout=layout),
                W.IntSlider(value=0, min=-3, max=3, step=1, description='d (chg)', style=style, layout=layout),
            ], layout=W.Layout(border='1px solid #ccc', padding='6px', margin='4px')),
            W.VBox([
                W.HTML("<b>δ13 = (p·n + q·k + r·c + s·sign(charge)) mod 13</b>"),
                W.IntSlider(value=1, min=-9, max=9, step=1, description='p (n)', style=style, layout=layout),
                W.IntSlider(value=1, min=-9, max=9, step=1, description='q (k)', style=style, layout=layout),
                W.IntSlider(value=2, min=-9, max=9, step=1, description='r (c)', style=style, layout=layout),
                W.IntSlider(value=0, min=-3, max=3, step=1, description='s (chg)', style=style, layout=layout),
            ], layout=W.Layout(border='1px solid #ccc', padding='6px', margin='4px')),
        ]),
        W.HBox([
            W.VBox([
                W.FloatText(value=1.0, description='tau₀ (сек)', style=style, layout=layout),
                W.Checkbox(value=USE_PDG_APPROX, description='Исп. τ (прибл.)', layout=layout),
                W.IntText(value=2, description='c_guess (канд.)', style=style, layout=layout),
            ]),
            W.VBox([
                W.IntRangeSlider(value=[-8,8], min=-16, max=16, step=1, description='n range', style=style, layout=W.Layout(width='420px')),
                W.IntRangeSlider(value=[0,16], min=0, max=24, step=1, description='k range', style=style, layout=W.Layout(width='420px')),
                W.Checkbox(value=True, description='Показывать графики', layout=layout),
            ]),
        ]),
    ])

    def _runner(a_n, b_k, c_c, d_chg, p_n, q_k, r_c, s_chg, tau0_val, use_pdg, c_guess_val, n_rng, k_rng, show_plots):
        clear_output(wait=True)
        display(ui)
        df, cand = run(chi_a=a_n, chi_b=b_k, chi_c=c_c, chi_d=d_chg,
                       del_p=p_n, del_q=q_k, del_r=r_c, del_s=s_chg,
                       tau0=tau0_val, use_pdg=use_pdg, c_guess=c_guess_val,
                       n_min=n_rng[0], n_max=n_rng[1], k_min=k_rng[0], k_max=k_rng[1],
                       show_plots=show_plots)

    ctrl = W.interactive_output(
        _runner,
        {
            # χ
            'a_n': ui.children[1].children[0].children[1],
            'b_k': ui.children[1].children[0].children[2],
            'c_c': ui.children[1].children[0].children[3],
            'd_chg': ui.children[1].children[0].children[4],
            # δ
            'p_n': ui.children[1].children[1].children[1],
            'q_k': ui.children[1].children[1].children[2],
            'r_c': ui.children[1].children[1].children[3],
            's_chg': ui.children[1].children[1].children[4],
            # misc
            'tau0_val': ui.children[2].children[0].children[0],
            'use_pdg': ui.children[2].children[0].children[1],
            'c_guess_val': ui.children[2].children[0].children[2],
            'n_rng': ui.children[2].children[1].children[0],
            'k_rng': ui.children[2].children[1].children[1],
            'show_plots': ui.children[2].children[1].children[2],
        }
    )
    display(ui, ctrl)
else:
    # Без виджетов — один запуск с настройками по умолчанию
    _ = run(
        chi_a=2, chi_b=1, chi_c=1, chi_d=0,
        del_p=1, del_q=1, del_r=2, del_s=0,
        tau0=1.0, use_pdg=USE_PDG_APPROX, c_guess=2,
        n_min=-8, n_max=8, k_min=0, k_max=16,
        show_plots=True
    )

In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# EN-suite (вторая ячейка): углублённые гипотезы для neutron & electron
# Ключевая связка "13/8" = δ₁₃ (door-класс) + ρ₈ (электро-класс)
# Ничего не сохраняет, не ломает первую ячейку.
# ============================================================

import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt

# ---- безоп. доступ к объектам из первой ячейки ----
PHI = (1 + 5**0.5) / 2.0
try:
    ME = ME
except NameError:
    ME = 0.510999

def _has(name): return name in globals()

# --- вспомогательные (на случай, если нет из 1-й ячейки) ---
def _mass_pred_local(n,k):
    BASE = PHI**PHI
    return ME * (2.0**n) * (BASE**k)

if not _has('mass_pred'):
    mass_pred = _mass_pred_local

# ---------- ПРЕСЕТЫ EN (13/8) ----------
# χ11 = (a·n + b·k + c·c + d·sign(q)) mod 11  → хотим Δχ11(p,n)=±1
CHI_COEFFS = (2, 1, 1, 1)     # a,b,c,d

# δ13 = (p·n + q·k + r·c + s·sign(q)) mod 13  → хотим Δδ13(p,n)=5
DELTA_COEFFS = (1, 1, 2, 5)   # p,q,r,s  (s=5 даёт Δ=5 для p(+1) vs n(0))

# ρ8 — «электронная» проекция (связка с 8 как рациональной аппроксимацией φ: 13/8≈1.625)
# берём простую целочисл. линейную форму:
RHO8_COEFFS = (2, 1, 1, 1)    # u·n + v·k + w·c + x·sign(q) mod 8

# --- калибровка τ0 для door φ⁵: λ(n)=5 ---
def _auto_tau0_for_neutron_phi5(tau_n_seconds):
    return tau_n_seconds / (PHI**5)

# ---------- построение df с нужными индикаторами ----------
def build_df_en_suite(chi=CHI_COEFFS, delta=DELTA_COEFFS, rho8=RHO8_COEFFS,
                      tau0=None, use_pdg=True):
    """Опираемся на build_df из 1-й ячейки, если есть; иначе — минимальная локальная сборка."""
    # попробуем извлечь τ нейтрона из первой ячейки
    tau_neutron = None
    if _has('tau_approx') and isinstance(tau_approx, dict) and 'neutron' in tau_approx:
        tau_neutron = float(tau_approx['neutron'])
    if tau0 is None and tau_neutron and math.isfinite(tau_neutron):
        tau0 = _auto_tau0_for_neutron_phi5(tau_neutron)  # даём λ(n)≈5
    if tau0 is None:
        tau0 = 79.4  # безопасный дефолт, если нет данных

    # используем build_df из 1-й ячейки
    if _has('build_df'):
        df = build_df(chi_coeffs=chi, delta_coeffs=delta, tau0=tau0, use_pdg=use_pdg)
    else:
        # fallback: минимальная реконструкция из df0
        if not _has('df0'):
            raise RuntimeError("Нет df0/build_df из первой ячейки. Запусти первую ячейку.")
        a,b,c,d = chi; p,q,r,s = delta
        df = df0.copy()
        df['K'] = df['n'].abs() + df['k'].abs()
        df['k_mod6'] = df['k'] % 6
        BASE = PHI**PHI
        df['mass_pred_MeV'] = df.apply(lambda r: ME*(2.0**r['n'])*(BASE**r['k']), axis=1)
        df['rel_err_%'] = (df['mass_pred_MeV'] - df['mass_MeV']).abs() / df['mass_MeV'] * 100.0
        # lifetimes, если доступны
        if _has('tau_approx'):
            df['tau_s'] = df['name'].map(tau_approx).astype(float)
        else:
            df['tau_s'] = np.nan
        df['lambda12'] = np.where(np.isfinite(df['tau_s']),
                                  np.round(np.log(df['tau_s']/tau0)/np.log(PHI)).astype('float'),
                                  np.nan)
        df['chi11']   = (a*df['n'] + b*df['k'] + c*df['c'] + d*np.sign(df['charge'])) % 11
        df['delta13'] = (p*df['n'] + q*df['k'] + r*df['c'] + s*np.sign(df['charge'])) % 13

    # ρ8
    u,v,w,x = rho8
    df['rho8'] = (u*df['n'] + v*df['k'] + w*df['c'] + x*np.sign(df['charge'])) % 8
    df['k_parity'] = df['k'] % 2
    return df, tau0

# ---------- функции вычисления ρ8/δ для пустых узлов ----------
def rho8_of(n, k, c_guess=2, sign_q=0, rho8_coeffs=RHO8_COEFFS):
    u,v,w,x = rho8_coeffs
    return (u*n + v*k + w*c_guess + x*sign_q) % 8

def delta13_of(n, k, c_guess=2, sign_q=0, delta_coeffs=DELTA_COEFFS):
    p,q,r,s = delta_coeffs
    return (p*n + q*k + r*c_guess + s*sign_q) % 13

# ---------- кандидаты под двойной ключ (δ13, ρ8) ----------
def dual_key_candidates(df, n_rng=(-8,8), k_rng=(0,16),
                        key_particle='neutron', c_guess=2, sign_q=0,
                        top=30):
    if not _has('occupied_empty_nodes'):
        raise RuntimeError("Нет occupied_empty_nodes из первой ячейки. Запусти первую ячейку.")
    occ, empty, _ = occupied_empty_nodes(df, n_min=n_rng[0], n_max=n_rng[1],
                                         k_min=k_rng[0], k_max=k_rng[1])
    d_key = int(df.loc[df['name']==key_particle,'delta13'].iloc[0])
    r_key = int(df.loc[df['name']==key_particle,'rho8'].iloc[0])

    rows = []
    for (n,k) in empty:
        K = abs(n)+abs(k)
        d = delta13_of(n,k,c_guess=c_guess, sign_q=sign_q)
        r = rho8_of(n,k,c_guess=c_guess, sign_q=sign_q)
        if (d == d_key) and (r == r_key):
            mp = mass_pred(n,k)
            rows.append((K,n,k,d,r,mp))
    cdf = pd.DataFrame(rows, columns=['K','n','k','delta13','rho8','mass_pred_MeV']) \
            .sort_values(['K','mass_pred_MeV']).head(top)
    return cdf

# ================== ЗАПУСК EN-suite ==================
df_en, tau0_used = build_df_en_suite()

# ключи и проверки
p = df_en[df_en['name']=='proton' ].iloc[0]
n = df_en[df_en['name']=='neutron'].iloc[0]
e = df_en[df_en['name']=='electron'].iloc[0]

dchi_np = (int(n['chi11']) - int(p['chi11'])) % 11
ddel_np = (int(n['delta13']) - int(p['delta13'])) % 13

print("="*90)
print("EN-suite: пресет (13/8)")
print("="*90)
print(f"χ₁₁ coeffs = {CHI_COEFFS}  → χ₁₁(p)={int(p['chi11'])}, χ₁₁(n)={int(n['chi11'])}, Δχ₁₁={dchi_np} (mod 11)")
print(f"δ₁₃ coeffs = {DELTA_COEFFS} → δ₁₃(p)={int(p['delta13'])}, δ₁₃(n)={int(n['delta13'])}, Δδ₁₃={ddel_np} (mod 13)")
if df_en['lambda12'].notna().any():
    print(f"λ(neutron)={df_en.loc[df_en['name']=='neutron','lambda12'].iloc[0]}  (τ₀≈{tau0_used:.3f} s; цель: 5)")
else:
    print(f"λ(neutron) недоступно (нет τ); принятый τ₀≈{tau0_used:.3f} s")

print("\nКлюч электрона (δ₁₃/ρ₈):")
print(f"  δ₁₃(e)={int(e['delta13'])}, ρ₈(e)={int(e['rho8'])}")
print("Ключ нейтрона (δ₁₃/ρ₈):")
print(f"  δ₁₃(n)={int(n['delta13'])}, ρ₈(n)={int(n['rho8'])}")

# ================== ДВОЙНОЙ ПОИСК КАНДИДАТОВ ==================
# 1) Нейтрон-класс (барионная гипотеза): c≈2, sign(q)=0
cand_neutron = dual_key_candidates(df_en, n_rng=(-8,8), k_rng=(0,16),
                                   key_particle='neutron', c_guess=2, sign_q=0, top=30)

# 2) Электрон-класс (лептонная гипотеза): c≈1, sign(q)=-1
cand_electron = dual_key_candidates(df_en, n_rng=(-8,8), k_rng=(0,16),
                                    key_particle='electron', c_guess=1, sign_q=-1, top=30)

print("\n" + "="*90)
print("КАНДИДАТЫ (двойной ключ δ₁₃ & ρ₈): Нейтрон-класс (c≈2, q≈0)")
print("="*90)
print(cand_neutron.to_string(index=False) if len(cand_neutron) else "— нет в заданной области.")

print("\n" + "="*90)
print("КАНДИДАТЫ (двойной ключ δ₁₃ & ρ₈): Электрон-класс (c≈1, q≈−1)")
print("="*90)
print(cand_electron.to_string(index=False) if len(cand_electron) else "— нет в заданной области.")

# ================== ВИЗУАЛИЗАЦИЯ (компакт) ==================
plt.rcParams['figure.figsize'] = (18, 6)
fig, axs = plt.subplots(1, 3)

# A) Карта (n,k) с подсветкой e/n + кандидаты по двойному ключу
families = df_en['family'].unique().tolist()
fam_colors = {'lepton':'#FF6B6B', 'quark':'#4ECDC4', 'baryon':'#45B7D1', 'meson':'#FFA07A', 'boson':'#98D8C8'}

ax = axs[0]
for f in families:
    dff = df_en[df_en['family']==f]
    ax.scatter(dff['n'], dff['k'], s=130, alpha=0.7,
               edgecolors='black', linewidth=1.2, c=fam_colors.get(f,'#999'), label=f)
ax.scatter(e['n'], e['k'], s=260, marker='s', c='yellow', edgecolors='black', linewidth=2, label='electron')
ax.scatter(n['n'], n['k'], s=260, marker='D', c='lime', edgecolors='black', linewidth=2, label='neutron')
if len(cand_neutron):
    ax.scatter(cand_neutron['n'], cand_neutron['k'], s=(cand_neutron['K']*35).clip(60, 250),
               marker='*', c='gold', edgecolors='blue', linewidth=1.5, label='cand. (neutron-key)')
if len(cand_electron):
    ax.scatter(cand_electron['n'], cand_electron['k'], s=(cand_electron['K']*35).clip(60, 250),
               marker='P', c='violet', edgecolors='black', linewidth=1.5, label='cand. (electron-key)')
ax.set_title('φ-решётка: e/n и кандидаты по двойному ключу (13/8)')
ax.set_xlabel('n'); ax.set_ylabel('k'); ax.grid(True, alpha=0.3)
ax.legend(fontsize=8)

# B) Распределения ключей
ax = axs[1]
bins13 = sorted(df_en['delta13'].unique())
bins08 = sorted(df_en['rho8'].unique())
ax.bar(np.arange(len(bins13)) - 0.2, df_en['delta13'].value_counts().reindex(bins13, fill_value=0), width=0.4, label='δ₁₃')
ax.bar(np.arange(len(bins08)) + 0.2, df_en['rho8'].value_counts().reindex(bins08,  fill_value=0), width=0.4, label='ρ₈')
ax.set_xticks(range(max(len(bins13),len(bins08))))
ax.set_xticklabels(range(max(len(bins13),len(bins08))))
ax.set_title('Частоты δ₁₃ и ρ₈')
ax.grid(True, axis='y', alpha=0.3); ax.legend()

# C) λ vs K (если есть τ)
ax = axs[2]
if df_en['lambda12'].notna().any():
    for f in families:
        dff = df_en[(df_en['family']==f) & (df_en['lambda12'].notna())]
        ax.scatter(dff['K'], dff['lambda12'], s=120, alpha=0.75,
                   edgecolors='black', linewidth=1.1, c=fam_colors.get(f,'#999'), label=f)
    # подписи для e/n
    if pd.notna(df_en.loc[df_en['name']=='neutron','lambda12'].iloc[0]):
        ax.annotate('neutron', (n['K'], n['lambda12']), fontsize=10, fontweight='bold')
    if pd.notna(df_en.loc[df_en['name']=='electron','lambda12'].iloc[0]):
        ax.annotate('electron', (e['K'], e['lambda12']), fontsize=10, fontweight='bold')
    ax.set_title('λ₁₂ (лог-φ τ) vs K'); ax.set_xlabel('K'); ax.set_ylabel('λ₁₂'); ax.grid(True, alpha=0.3)
else:
    ax.axis('off'); ax.text(0.5,0.5, "λ₁₂ недоступно (нет τ)", ha='center', va='center')

plt.tight_layout(); plt.show()

# ================== КОРОТКО: ГИПОТЕЗЫ 13/8 ==================
print("\n" + "="*90)
print("Гипотезы 13/8 (операциональные формулировки)")
print("="*90)
print("H13/8-A  (нейтрон-ключ): частицы реального «ядерного интерфейса» принадлежат классу (δ₁₃, ρ₈) нейтрона.")
print("H13/8-B  (электрон-ключ): электрон задаёт минимальный «EM-ключ» (δ₁₃, ρ₈) для лептонного семейства.")
print("H13/8-C  (door φ⁵): при τ₀=τ_n/φ⁵ имеем λ(neutron)≈5; любые отклонения → кандидаты на субструктуру/вмешательство.")
print("H13/8-D  (двойной отбор): узлы решётки, совпадающие и по δ₁₃, и по ρ₈ с e/n-ключами, — первоочередные к проверке.")

In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# ТРЕТЬЯ ЯЧЕЙКА — 13/8-СКАНЕР (устойчивость и мосты e↔n)
# Ничего не перезаписывает: опирается на объекты из 1–2 ячеек.
# ============================================================

import math, numpy as np, pandas as pd
import matplotlib.pyplot as plt

PHI = (1 + 5**0.5) / 2.0

# ---------- безопасные геттеры ----------
def _has(name): return name in globals()

def _df_en_ready():
    if _has('df_en'):
        return df_en
    elif _has('build_df_en_suite'):
        # построим по дефолту второй ячейки (λ(n)=5 уже калибровано)
        dfe, _tau0 = build_df_en_suite()
        return dfe
    elif _has('build_df'):
        return build_df(chi_coeffs=(2,1,1,1), delta_coeffs=(1,1,2,5), tau0=79.4, use_pdg=True)
    else:
        raise RuntimeError("Нужна хотя бы 1-я ячейка (df0/build_df) или 2-я (build_df_en_suite).")

def mass_pred_safe(n,k):
    if _has('mass_pred'):
        return mass_pred(n,k)
    BASE = PHI**PHI
    ME = 0.510999
    return ME * (2.0**n) * (BASE**k)

# ---------- δ13 и ρ8 с параметрами ----------
def delta13_of(n, k, c, sign_q, p=1, q=1, r=2, s=5):
    return (p*n + q*k + r*c + s*sign_q) % 13

def rho8_of(n, k, c, sign_q, u=2, v=1, w=1, x=1):
    return (u*n + v*k + w*c + x*sign_q) % 8

# ---------- подготовка набора ----------
df_base = _df_en_ready().copy()
p_row = df_base[df_base['name']=='proton' ].iloc[0]
n_row = df_base[df_base['name']=='neutron'].iloc[0]
e_row = df_base[df_base['name']=='electron'].iloc[0]

# Зафиксируем «геометрию» из предыдущих ячеек
chi_coeffs = (2,1,1,1)   # χ11
delta_pqr = (1,1,2)      # (p,q,r)
rho_uvwx   = (2,1,1,1)   # (u,v,w,x)

# ---------- СКАН ПО ВЕСАМ ЗАРЯДА: s для δ, x для ρ ----------
s_grid = list(range(1,13))      # δ-вес по знаку заряда
x_grid = list(range(0,8))       # ρ-вес по знаку заряда
records = []

for s in s_grid:
    d_p = delta13_of(p_row['n'], p_row['k'], p_row['c'], np.sign(p_row['charge']),
                     p=delta_pqr[0], q=delta_pqr[1], r=delta_pqr[2], s=s)
    d_n = delta13_of(n_row['n'], n_row['k'], n_row['c'], np.sign(n_row['charge']),
                     p=delta_pqr[0], q=delta_pqr[1], r=delta_pqr[2], s=s)
    d_diff = (int(d_n) - int(d_p)) % 13  # хотим 8
    for x in x_grid:
        r_e = rho8_of(e_row['n'], e_row['k'], e_row['c'], np.sign(e_row['charge']),
                      u=rho_uvwx[0], v=rho_uvwx[1], w=rho_uvwx[2], x=x)
        r_n = rho8_of(n_row['n'], n_row['k'], n_row['c'], np.sign(n_row['charge']),
                      u=rho_uvwx[0], v=rho_uvwx[1], w=rho_uvwx[2], x=x)
        # хотим ρ8(e)=0, ρ8(n)=6 (как в твоём прогоне)
        ok_r = (int(r_e)==0 and int(r_n)==6)
        records.append((s,x,int(d_diff),int(r_e),int(r_n),ok_r))

scan = pd.DataFrame(records, columns=['s_delta','x_rho','Δδ13','ρ8(e)','ρ8(n)','ρ_ok'])
best = scan[(scan['Δδ13']==8) & (scan['ρ_ok'])]

print("="*90)
print("Скан по (s, x): устойчивые настройки для «13/8»")
print("="*90)
if len(best)==0:
    print("Нет точного совпадения Δδ13=8 при ρ8(e)=0 и ρ8(n)=6. Показываю ближайшие по Δδ13.")
    print(scan.sort_values(['ρ_ok','Δδ13'], ascending=[False, True]).head(10).to_string(index=False))
else:
    print(best.head(10).to_string(index=False))
    s_star = int(best.iloc[0]['s_delta'])
    x_star = int(best.iloc[0]['x_rho'])

    # ---------- Собираем рабочий df с выбранными (s*, x*) ----------
    df = df_base.copy()
    p,q,r = delta_pqr
    u,v,w = rho_uvwx[:3]
    df['delta13'] = (p*df['n'] + q*df['k'] + r*df['c'] + s_star*np.sign(df['charge'])) % 13
    df['rho8']    = (u*df['n'] + v*df['k'] + w*df['c'] + x_star*np.sign(df['charge'])) % 8
    df['K'] = (df['n'].abs() + df['k'].abs()).astype(int)

    # ключи e/n (для печати)
    d_e, r_e = int(df.loc[df['name']=='electron','delta13']), int(df.loc[df['name']=='electron','rho8'])
    d_n, r_n = int(df.loc[df['name']=='neutron','delta13']), int(df.loc[df['name']=='neutron','rho8'])
    d_p      = int(df.loc[df['name']=='proton','delta13'])
    print(f"\nКлючи при (s={s_star}, x={x_star}):")
    print(f"  e: δ={d_e}, ρ={r_e} | n: δ={d_n}, ρ={r_n} | p: δ={d_p}")
    print(f"  Δδ13(n,p)={(d_n - d_p) % 13} (mod 13)  — цель 8 ✓")

    # ---------- Пустые узлы и три набора кандидатов ----------
    if not _has('occupied_empty_nodes'):
        from math import inf
        def occupied_empty_nodes(df, n_min=-8, n_max=8, k_min=0, k_max=16):
            occupied = set(zip(df['n'], df['k']))
            all_nodes = [(n,k) for n in range(n_min, n_max+1) for k in range(k_min, k_max+1)]
            empty = [(n,k) for (n,k) in all_nodes if (n,k) not in occupied]
            return occupied, empty, all_nodes

    occ, empty, _ = occupied_empty_nodes(df, n_min=-10, n_max=10, k_min=0, k_max=18)

    def cand_filter(target_delta, target_rho, c_guess=2, sign_q=0, top=30):
        rows=[]
        for (n,k) in empty:
            K = abs(n)+abs(k)
            d = delta13_of(n,k,c=c_guess, sign_q=sign_q, p=p,q=q,r=r,s=s_star)
            r8= rho8_of(n,k,c=c_guess, sign_q=sign_q, u=u,v=v,w=w,x=x_star)
            if (d==target_delta) and (r8==target_rho):
                rows.append((K,n,k,int(d),int(r8), mass_pred_safe(n,k)))
        cdf = pd.DataFrame(rows, columns=['K','n','k','delta13','rho8','mass_pred_MeV']).sort_values(['K','mass_pred_MeV'])
        return cdf.head(top)

    # Нейтрон-класс (c≈2, q≈0)
    cand_N = cand_filter(d_n, r_n, c_guess=2, sign_q=0, top=50)
    # Электрон-класс (c≈1, q≈−1)
    cand_E = cand_filter(d_e, r_e, c_guess=1, sign_q=-1, top=50)
    # МОСТЫ: δ как у n, ρ как у e (и наоборот)
    cand_bridge_NE = cand_filter(d_n, r_e, c_guess=2, sign_q=0, top=50)
    cand_bridge_EN = cand_filter(d_e, r_n, c_guess=1, sign_q=-1, top=50)

    print("\nТОП кандидатов — нейтрон-класс (δ/ρ как у n):")
    print(cand_N.head(15).to_string(index=False) if len(cand_N) else "— нет в области поиска.")
    print("\nТОП кандидатов — электрон-класс (δ/ρ как у e):")
    print(cand_E.head(15).to_string(index=False) if len(cand_E) else "— нет в области поиска.")
    print("\nМОСТ (δ=δ(n), ρ=ρ(e)) — интерфейс e↔n, барионный c≈2:")
    print(cand_bridge_NE.head(15).to_string(index=False) if len(cand_bridge_NE) else "— нет.")
    print("\nМОСТ (δ=δ(e), ρ=ρ(n)) — обратный интерфейс:")
    print(cand_bridge_EN.head(15).to_string(index=False) if len(cand_bridge_EN) else "— нет.")

    # ---------- УСТОЙЧИВОСТЬ КАНДИДАТОВ ----------
    # Пройдём по всем (s,x), где Δδ13=8 и ρ-ключи совпадают, и накопим частоты появления узла.
    occ_counts = {}
    good_pairs = best[['s_delta','x_rho']].drop_duplicates().values.tolist()
    for s_try, x_try in good_pairs:
        for (n,k) in empty:
            d = delta13_of(n,k,c=2, sign_q=0, p=p,q=q,r=r,s=s_try)
            r8= rho8_of(n,k,c=2, sign_q=0, u=u,v=v,w=w,x=x_try)
            if d==d_n and r8==r_n:
                occ_counts[(n,k)] = occ_counts.get((n,k),0) + 1

    if good_pairs:
        total_pairs = len(good_pairs)
        stable = [(n,k,cnt, mass_pred_safe(n,k), abs(n)+abs(k)) for (n,k),cnt in occ_counts.items() if cnt/total_pairs>=0.8]
        stable_df = pd.DataFrame(stable, columns=['n','k','stability_hits','mass_pred_MeV','K']).sort_values(['K','stability_hits','mass_pred_MeV'])
        print(f"\nСТАБИЛЬНЫЕ кандидаты (нейтрон-ключ), устойчивость ≥80% по сетке ({total_pairs} конфигураций):")
        print(stable_df.head(20).to_string(index=False) if len(stable_df) else "— не найдено устойчивых в этой области.")

    # ---------- ВИЗУАЛИЗАЦИЯ ----------
    plt.rcParams['figure.figsize'] = (18, 6)
    fig, axs = plt.subplots(1,3)

    # A) δ–ρ-облака по семействам + e/n
    fam_colors = {'lepton':'#FF6B6B','quark':'#4ECDC4','baryon':'#45B7D1','meson':'#FFA07A','boson':'#98D8C8'}
    ax = axs[0]
    for f in df['family'].unique():
        dff = df[df['family']==f]
        ax.scatter(dff['delta13'], dff['rho8'], s=120, alpha=0.7,
                   edgecolors='black', linewidth=1.1, c=fam_colors.get(f,'#999'), label=f)
    ax.scatter(d_e, r_e, s=260, marker='s', c='yellow', edgecolors='black', linewidth=2, label='electron')
    ax.scatter(d_n, r_n, s=260, marker='D', c='lime',   edgecolors='black', linewidth=2, label='neutron')
    ax.set_xticks(range(13)); ax.set_xlim(-0.5,12.5)
    ax.set_yticks(range(8));  ax.set_ylim(-0.5,7.5)
    ax.set_xlabel('δ₁₃'); ax.set_ylabel('ρ₈'); ax.set_title('Карта ключей (δ₁₃, ρ₈)')
    ax.grid(True, alpha=0.3); ax.legend(fontsize=8)

    # B) Частоты δ и ρ
    ax = axs[1]
    bins13 = range(13); bins8 = range(8)
    ax.bar([x-0.2 for x in bins13], df['delta13'].value_counts().reindex(bins13, fill_value=0), width=0.4, label='δ₁₃')
    ax.bar([x+0.2 for x in bins8],  df['rho8'   ].value_counts().reindex(bins8,  fill_value=0), width=0.4, label='ρ₈')
    ax.set_title('Частоты δ₁₃ и ρ₈'); ax.grid(True, axis='y', alpha=0.3); ax.legend()

    # C) Стабильность кандидатов (если есть)
    ax = axs[2]
    if good_pairs and occ_counts:
        items = sorted(occ_counts.items(), key=lambda kv: (-kv[1], abs(kv[0][0])+abs(kv[0][1])))
        xs = list(range(min(len(items),15)))
        labs = [f"({n},{k})" for (n,k),_ in items[:15]]
        vals = [cnt/len(good_pairs)*100 for _,cnt in items[:15]]
        ax.bar(xs, vals, edgecolor='black')
        ax.set_xticks(xs); ax.set_xticklabels(labs, rotation=45)
        ax.set_ylim(0,100); ax.set_ylabel('% устойчивости')
        ax.set_title('Топ-узлы по устойчивости (нейтрон-ключ)')
        ax.grid(True, axis='y', alpha=0.3)
    else:
        ax.axis('off'); ax.text(0.5,0.5,"Нет устойчивых наборов (проверь условия сканера)", ha='center', va='center')

    plt.tight_layout(); plt.show()

In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# 4-я ячейка: EN-report 13/8 (фильтры, чистый вывод, без CSV)
# ============================================================

import numpy as np, pandas as pd, math
import matplotlib.pyplot as plt

PHI = (1 + 5**0.5) / 2.0

# ---- безопасные геттеры из предыдущих ячеек ----
def _has(name): return name in globals()

def mass_pred_safe(n,k):
    if _has('mass_pred'):
        return mass_pred(n,k)
    BASE = PHI**PHI
    ME = 0.510999
    return ME * (2.0**n) * (BASE**k)

def df_en_ready():
    if _has('df_en'):
        return df_en.copy()
    elif _has('build_df_en_suite'):
        dfe, _ = build_df_en_suite()
        return dfe
    elif _has('build_df'):
        return build_df(chi_coeffs=(2,1,1,1), delta_coeffs=(1,1,2,5), tau0=79.4, use_pdg=True)
    else:
        raise RuntimeError("Нужна хотя бы 1-я ячейка (df0/build_df) или 2-я (build_df_en_suite).")

# ---- ключевые коэффициенты 13/8 (из устойчивого скана) ----
# χ11 держим (2,1,1,1) из 2-й ячейки.
DELTA_S_STAR = 5   # s (вес заряда в δ) — даёт Δδ13(n,p)=8
RHO_X_STAR   = 1   # x (вес заряда в ρ) — даёт ρ8(e)=0, ρ8(n)=6

# ---- фильтры отчёта (можешь менять прямо тут) ----
ONLY_N_GE_0_BARYONS = True      # для барионных кандидатов требовать n ≥ 0
MASS_WINDOWS_MEV = {            # «интересные окна» для подсветки (можно пустыми оставить)
    "pion~140":   (100, 200),
    "kaon~500":   (420, 560),
    "nucleon~940":(800, 1100),
    "J/psi~3097": (2800, 3400),
}
TOP_SHOW = 20                    # сколько показывать в каждой корзине
N_RANGE = (-10, 10)              # диапазон сканирования узлов n
K_RANGE = (0, 18)                # диапазон сканирования узлов k
C_GUESS = dict(neutron=2, electron=1)  # предполагаемый c при поиске узлов

# ---- δ13 и ρ8 с параметрами (совместимо с прежними формулами) ----
def delta13_of(n, k, c, sign_q, p=1, q=1, r=2, s=DELTA_S_STAR):
    return (p*n + q*k + r*c + s*sign_q) % 13

def rho8_of(n, k, c, sign_q, u=2, v=1, w=1, x=RHO_X_STAR):
    return (u*n + v*k + w*c + x*sign_q) % 8

# ---- построение рабочего df с δ/ρ (фиксируем s*, x*) ----
df = df_en_ready()
p_row = df.loc[df['name']=='proton' ].iloc[0]
n_row = df.loc[df['name']=='neutron'].iloc[0]
e_row = df.loc[df['name']=='electron'].iloc[0]

# Пересчитываем δ/ρ по (s*, x*) независимо от того, что было раньше, чтобы всё согласовать
p,q,r = 1,1,2; u,v,w = 2,1,1
df['delta13'] = (p*df['n'] + q*df['k'] + r*df['c'] + DELTA_S_STAR*np.sign(df['charge'])) % 13
df['rho8']    = (u*df['n'] + v*df['k'] + w*df['c'] + RHO_X_STAR*np.sign(df['charge'])) % 8
df['K']       = (df['n'].abs() + df['k'].abs()).astype(int)

# ключи (без FutureWarning)
d_e = int(df.loc[df['name']=='electron','delta13'].iloc[0]); r_e = int(df.loc[df['name']=='electron','rho8'].iloc[0])
d_n = int(df.loc[df['name']=='neutron','delta13'].iloc[0]);  r_n = int(df.loc[df['name']=='neutron','rho8'].iloc[0])
d_p = int(df.loc[df['name']=='proton','delta13'].iloc[0])

print("="*90)
print("EN-report 13/8 — ключи и фазы")
print("="*90)
print(f"  electron: δ={d_e}, ρ={r_e}")
print(f"  neutron : δ={d_n}, ρ={r_n}")
print(f"  proton  : δ={d_p}")
print(f"  Δδ13(n,p)={(d_n - d_p) % 13} (mod 13)  | ожидаем 8  ✓")
if 'lambda12' in df.columns and pd.notna(df.loc[df['name']=='neutron','lambda12'].iloc[0]):
    print(f"  λ(neutron)={df.loc[df['name']=='neutron','lambda12'].iloc[0]} (должно быть ≈5 при τ₀=τ_n/φ⁵)")

# ---- генератор пустых узлов ----
def occupied_empty_nodes_local(df, n_min, n_max, k_min, k_max):
    occupied = set(zip(df['n'], df['k']))
    all_nodes = [(n,k) for n in range(n_min, n_max+1) for k in range(k_min, k_max+1)]
    empty = [(n,k) for (n,k) in all_nodes if (n,k) not in occupied]
    return occupied, empty

occ, empty = occupied_empty_nodes_local(df, N_RANGE[0], N_RANGE[1], K_RANGE[0], K_RANGE[1])

# ---- фильтр и сбор кандидатов ----
def collect_candidates(target_delta, target_rho, c_guess, sign_q, family_hint=None, top=TOP_SHOW, n_ge_0=False):
    rows=[]
    for (n,k) in empty:
        if n_ge_0 and n < 0:
            continue
        # целевые проекции
        d = delta13_of(n,k,c=c_guess, sign_q=sign_q, p=p,q=q,r=r, s=DELTA_S_STAR)
        r8= rho8_of(n,k,c=c_guess, sign_q=sign_q, u=u,v=v,w=w, x=RHO_X_STAR)
        if d==target_delta and r8==target_rho:
            mp = mass_pred_safe(n,k)
            # попадание в окна масс (только подсветка)
            tag = []
            for name,(lo,hi) in MASS_WINDOWS_MEV.items():
                if lo <= mp <= hi:
                    tag.append(name)
            rows.append((abs(n)+abs(k), n, k, int(d), int(r8), mp, ",".join(tag)))
    cdf = pd.DataFrame(rows, columns=['K','n','k','delta13','rho8','mass_pred_MeV','in_window']) \
           .sort_values(['K','mass_pred_MeV']).head(top)
    return cdf

# 1) Нейтрон-класс: c≈2, q≈0
cand_N = collect_candidates(d_n, r_n, c_guess=C_GUESS['neutron'], sign_q=0,
                            family_hint='baryon', top=TOP_SHOW, n_ge_0=ONLY_N_GE_0_BARYONS)

# 2) Электрон-класс: c≈1, q≈−1
cand_E = collect_candidates(d_e, r_e, c_guess=C_GUESS['electron'], sign_q=-1,
                            family_hint='lepton', top=TOP_SHOW, n_ge_0=False)

# 3) Мосты (δ=δ(n), ρ=ρ(e)) и (δ=δ(e), ρ=ρ(n))
cand_bridge_NE = collect_candidates(d_n, r_e, c_guess=C_GUESS['neutron'], sign_q=0,
                                    family_hint='baryon', top=TOP_SHOW, n_ge_0=ONLY_N_GE_0_BARYONS)
cand_bridge_EN = collect_candidates(d_e, r_n, c_guess=C_GUESS['electron'], sign_q=-1,
                                    family_hint='lepton', top=TOP_SHOW, n_ge_0=False)

# ---- печать отчёта ----
pd.set_option("display.float_format", lambda x: f"{x:,.6g}")

print("\nТОП кандидатов — нейтрон-класс (δ/ρ как у n, c≈2, q≈0):")
print(cand_N if len(cand_N) else "— нет в области поиска.")

print("\nТОП кандидатов — электрон-класс (δ/ρ как у e, c≈1, q≈−1):")
print(cand_E if len(cand_E) else "— нет в области поиска.")

print("\nМОСТ (δ=δ(n), ρ=ρ(e)) — интерфейс e↔n (c≈2, q≈0):")
print(cand_bridge_NE if len(cand_bridge_NE) else "— нет.")

print("\nМОСТ (δ=δ(e), ρ=ρ(n)) — обратный интерфейс (c≈1, q≈−1):")
print(cand_bridge_EN if len(cand_bridge_EN) else "— нет.")

# ---- компактная визуализация ----
plt.rcParams['figure.figsize'] = (18,6)
fig, axs = plt.subplots(1,3)

# A) δ–ρ карта ключей по семействам
fam_colors = {'lepton':'#FF6B6B','quark':'#4ECDC4','baryon':'#45B7D1','meson':'#FFA07A','boson':'#98D8C8'}
ax = axs[0]
for f in df['family'].unique():
    dff = df[df['family']==f]
    ax.scatter(dff['delta13'], dff['rho8'], s=120, alpha=0.7,
               edgecolors='black', linewidth=1.1, c=fam_colors.get(f,'#999'), label=f)
ax.scatter(d_e, r_e, s=260, marker='s', c='yellow', edgecolors='black', linewidth=2, label='electron')
ax.scatter(d_n, r_n, s=260, marker='D', c='lime',   edgecolors='black', linewidth=2, label='neutron')
ax.set_xticks(range(13)); ax.set_xlim(-0.5,12.5)
ax.set_yticks(range(8));  ax.set_ylim(-0.5,7.5)
ax.set_xlabel('δ₁₃'); ax.set_ylabel('ρ₈'); ax.set_title('Ключи (δ₁₃, ρ₈)')
ax.grid(True, alpha=0.3); ax.legend(fontsize=8)

# B) Кандидаты n/e (наносятся поверх существующих)
def _scatter_cand(ax, dfc, marker, color, label):
    if len(dfc):
        ax.scatter(dfc['delta13'], dfc['rho8'],
                   s=(dfc['K']*35).clip(60,250), marker=marker,
                   c=color, edgecolors='black', linewidth=1.4, alpha=0.9, label=label)

_scatter_cand(ax, cand_N,         '*', 'gold',   'cand N-key')
_scatter_cand(ax, cand_E,         'P', 'violet', 'cand E-key')
_scatter_cand(ax, cand_bridge_NE, 'X', 'cyan',   'bridge N→E')
_scatter_cand(ax, cand_bridge_EN, '^', 'pink',   'bridge E→N')

# C) Массовые окна: диаграмма попаданий
ax = axs[1]
labels, counts = [], []
for name,(lo,hi) in MASS_WINDOWS_MEV.items():
    cnt = 0
    for tbl in [cand_N, cand_E, cand_bridge_NE, cand_bridge_EN]:
        if len(tbl): cnt += (tbl['in_window']!='').sum()
    labels.append(name); counts.append(cnt)
ax.bar(labels, counts, edgecolor='black')
ax.set_title('Попадания кандидатов в заданные окна масс')
ax.grid(True, axis='y', alpha=0.3)

# D) K-распределение кандидатов
ax = axs[2]
all_cands = pd.concat([cand_N, cand_E, cand_bridge_NE, cand_bridge_EN], ignore_index=True) if len(cand_N) or len(cand_E) or len(cand_bridge_NE) or len(cand_bridge_EN) else pd.DataFrame(columns=['K'])
if len(all_cands):
    ax.hist(all_cands['K'], bins=range(int(all_cands['K'].min()), int(all_cands['K'].max())+2),
            edgecolor='black', alpha=0.8)
    ax.set_title('Распределение K среди кандидатов')
    ax.set_xlabel('K'); ax.set_ylabel('кол-во'); ax.grid(True, axis='y', alpha=0.3)
else:
    ax.axis('off'); ax.text(0.5,0.5,"Кандидатов нет в текущих фильтрах", ha='center', va='center')

plt.tight_layout(); plt.show()

In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# 5-я ячейка — Локальный 13/8-поиск + скоринг (n/e + мосты)
# Ничего не переопределяет; использует df из 2–4 ячеек.
# ============================================================

import numpy as np, pandas as pd, math
import matplotlib.pyplot as plt

PHI = (1 + 5**0.5) / 2.0

# ---------- безопасные геттеры ----------
def _has(name): return name in globals()

def mass_pred_safe(n,k):
    if _has('mass_pred'):
        return mass_pred(n,k)
    ME = 0.510999; BASE = PHI**PHI
    return ME * (2.0**n) * (BASE**k)

def df_en_ready():
    # возьмём согласованный df из 4-й ячейки, иначе из 2-й
    if _has('df'):
        return df.copy()
    if _has('df_en'):
        return df_en.copy()
    if _has('build_df_en_suite'):
        dfe, _ = build_df_en_suite()
        return dfe
    raise RuntimeError("Нужна хотя бы 2-я ячейка (build_df_en_suite).")

# ---------- зафиксированные ключи 13/8 ----------
DELTA_S_STAR = 5  # вес заряда в δ (из сканера)
RHO_X_STAR   = 1  # вес заряда в ρ (из сканера)

# δ13 = n + k + 2c + s*sign(q) (mod 13)
def delta13_of(n, k, c, sign_q, p=1, q=1, r=2, s=DELTA_S_STAR):
    return (p*n + q*k + r*c + s*sign_q) % 13

# ρ8  = 2n + k + c + x*sign(q) (mod 8)
def rho8_of(n, k, c, sign_q, u=2, v=1, w=1, x=RHO_X_STAR):
    return (u*n + v*k + w*c + x*sign_q) % 8

# ---------- настройки поиска ----------
N_RANGE = (-12, 12)
K_RANGE = (0, 20)

ONLY_N_GE_0_BARYONS = True
C_GUESS = dict(neutron=2, electron=1)

# окна масс (центр, полуширина)
MASS_WINDOWS = {
    "pion~140":    (140.0, 40.0),
    "kaon~500":    (495.0, 90.0),
    "nucleon~940": (938.0, 120.0),
    "J/psi~3097":  (3097.0, 350.0),
}

# веса скоринга
W_K       = 0.55   # минимальность K
W_WINDOW  = 0.30   # близость к окнам масс
W_CENTER  = 0.15   # близость к центру (2,8)
CENTER_NK = (2, 8)

def window_score(m_mev):
    if not MASS_WINDOWS:
        return 0.0
    best = 0.0
    for _, (c, half) in MASS_WINDOWS.items():
        d = abs(m_mev - c)
        s = max(0.0, 1.0 - d/half)  # внутри окна → 1..0
        best = max(best, s)
    return best  # 0..1

def score_candidate(n,k,m_mev,is_baryon=False):
    K = abs(n)+abs(k)
    sK = 1.0/(1.0 + K)                     # чем меньше K, тем лучше
    sW = window_score(m_mev)               # попадание в окна
    dC = abs(n-CENTER_NK[0]) + abs(k-CENTER_NK[1])
    sC = 1.0/(1.0 + dC)                    # ближе к (2,8) — лучше
    # штраф за n<0 для барионов (если включено)
    penalty = 0.8 if (is_baryon and ONLY_N_GE_0_BARYONS and n < 0) else 1.0
    return penalty * (W_K*sK + W_WINDOW*sW + W_CENTER*sC)

# ---------- подготовка базы ----------
df0 = df_en_ready()
p_row = df0.loc[df0['name']=='proton' ].iloc[0]
n_row = df0.loc[df0['name']=='neutron'].iloc[0]
e_row = df0.loc[df0['name']=='electron'].iloc[0]

# пересчёт δ/ρ по (s*, x*)
p,q,r = 1,1,2; u,v,w = 2,1,1
df0['delta13'] = (p*df0['n'] + q*df0['k'] + r*df0['c'] + DELTA_S_STAR*np.sign(df0['charge'])) % 13
df0['rho8']    = (u*df0['n'] + v*df0['k'] + w*df0['c'] + RHO_X_STAR*np.sign(df0['charge'])) % 8
d_e, r_e = int(df0.loc[df0['name']=='electron','delta13'].iloc[0]), int(df0.loc[df0['name']=='electron','rho8'].iloc[0])
d_n, r_n = int(df0.loc[df0['name']=='neutron','delta13'].iloc[0]),  int(df0.loc[df0['name']=='neutron','rho8'].iloc[0])

# ---------- генератор пустых узлов ----------
occupied = set(zip(df0['n'], df0['k']))
all_nodes = [(n,k) for n in range(N_RANGE[0], N_RANGE[1]+1) for k in range(K_RANGE[0], K_RANGE[1]+1)]
empty = [(n,k) for (n,k) in all_nodes if (n,k) not in occupied]

def collect_bucket(target_delta, target_rho, c_guess, sign_q, is_baryon=False, top=25):
    rows=[]
    for (n,k) in empty:
        if is_baryon and ONLY_N_GE_0_BARYONS and n < 0:
            continue
        d = delta13_of(n,k,c=c_guess, sign_q=sign_q)
        r8= rho8_of(n,k,c=c_guess, sign_q=sign_q)
        if d==target_delta and r8==target_rho:
            mp = mass_pred_safe(n,k)
            sc = score_candidate(n,k,mp,is_baryon=is_baryon)
            rows.append((sc, abs(n)+abs(k), n, k, int(d), int(r8), mp))
    cdf = pd.DataFrame(rows, columns=['score','K','n','k','delta13','rho8','mass_pred_MeV']) \
           .sort_values(['score','K'], ascending=[False,True]).head(top)
    return cdf

# корзины
cand_N   = collect_bucket(d_n, r_n, c_guess=C_GUESS['neutron'], sign_q=0,  is_baryon=True,  top=25)
cand_E   = collect_bucket(d_e, r_e, c_guess=C_GUESS['electron'], sign_q=-1, is_baryon=False, top=25)
bridge_NE= collect_bucket(d_n, r_e, c_guess=C_GUESS['neutron'], sign_q=0,  is_baryon=True,  top=25)
bridge_EN= collect_bucket(d_e, r_n, c_guess=C_GUESS['electron'], sign_q=-1, is_baryon=False, top=25)

# ---------- отчёт ----------
pd.set_option("display.float_format", lambda x: f"{x:,.6g}")
print("TOP (N-key):")
print(cand_N if len(cand_N) else "— нет.")
print("\nTOP (E-key):")
print(cand_E if len(cand_E) else "— нет.")
print("\nTOP (Bridge N→E):")
print(bridge_NE if len(bridge_NE) else "— нет.")
print("\nTOP (Bridge E→N):")
print(bridge_EN if len(bridge_EN) else "— нет.")

# ---------- визуализация ----------
plt.rcParams['figure.figsize'] = (19,6)
fig, axs = plt.subplots(1,3)

# A) карта (n,k) + пузырь по score
fam_colors = {'lepton':'#FF6B6B','quark':'#4ECDC4','baryon':'#45B7D1','meson':'#FFA07A','boson':'#98D8C8'}
ax = axs[0]
for f in df0['family'].unique():
    dff = df0[df0['family']==f]
    ax.scatter(dff['n'], dff['k'], s=120, alpha=0.55, edgecolors='black', linewidth=1.0,
               c=fam_colors.get(f,'#999'), label=f)
# кандидаты
def plot_bucket(ax, dfb, marker, color, label):
    if len(dfb):
        sizes = (dfb['score']*800).clip(60, 400)
        ax.scatter(dfb['n'], dfb['k'], s=sizes, marker=marker, c=color,
                   edgecolors='black', linewidth=1.4, alpha=0.9, label=label)
plot_bucket(ax, cand_N,    '*', 'gold',   'N-key')
plot_bucket(ax, cand_E,    'P', 'violet', 'E-key')
plot_bucket(ax, bridge_NE, 'X', 'cyan',   'bridge N→E')
plot_bucket(ax, bridge_EN, '^', 'pink',   'bridge E→N')
# e/n маркеры
e = df0[df0['name']=='electron'].iloc[0]; n = df0[df0['name']=='neutron'].iloc[0]
ax.scatter([e['n']],[e['k']], s=260, marker='s', c='yellow', edgecolors='black', linewidth=2, label='electron')
ax.scatter([n['n']],[n['k']], s=260, marker='D', c='lime',   edgecolors='black', linewidth=2, label='neutron')
ax.set_xlabel('n'); ax.set_ylabel('k'); ax.set_title('Локальный 13/8-поиск: карта (n,k)')
ax.grid(True, alpha=0.3); ax.legend(fontsize=8)

# B) попадания в окна (по всем корзинам)
ax = axs[1]
labels, counts = [], []
for name,(c,half) in MASS_WINDOWS.items():
    cnt = 0
    for tbl in [cand_N, cand_E, bridge_NE, bridge_EN]:
        if len(tbl):
            # считаем попадание, если |m-center| <= half
            m = tbl['mass_pred_MeV'].values
            cnt += int(np.sum(np.abs(m - c) <= half))
    labels.append(name); counts.append(cnt)
ax.bar(labels, counts, edgecolor='black')
ax.set_title('Попадания кандидатов в заданные окна масс')
ax.grid(True, axis='y', alpha=0.3)

# C) распределение K (по всем корзинам)
ax = axs[2]
all_c = pd.concat([cand_N, cand_E, bridge_NE, bridge_EN], ignore_index=True) if any(len(t)>0 for t in [cand_N, cand_E, bridge_NE, bridge_EN]) else pd.DataFrame(columns=['K'])
if len(all_c):
    ax.hist(all_c['K'], bins=range(int(all_c['K'].min()), int(all_c['K'].max())+2), edgecolor='black', alpha=0.85)
    ax.set_title('Распределение K среди найденных кандидатов')
    ax.set_xlabel('K'); ax.set_ylabel('кол-во'); ax.grid(True, axis='y', alpha=0.3)
else:
    ax.axis('off'); ax.text(0.5,0.5,"кандидатов нет (проверь фильтры)", ha='center', va='center')

plt.tight_layout(); plt.show()

In [ ]:
# -*- coding: utf-8 -*-
# ============================================================
# 6-я ячейка — Тонкий 13/8-поиск: локальный скан и жёсткие фильтры
# - учитывает предыдущие ячейки; ничего не переопределяет глобально
# - для барионов: n>=0 и k чётный (по наблюдённому паттерну)
# - локально сканирует окрестности (2,8), (4,6), (10,0) и то, что ты укажешь
# - усиливает скоринг под окна масс и близость к центру
# ============================================================

import numpy as np, pandas as pd, math
import matplotlib.pyplot as plt

PHI = (1 + 5**0.5) / 2.0
DELTA_S_STAR = 5  # из стабильного скана
RHO_X_STAR   = 1  # из стабильного скана

def _has(name): return name in globals()

def mass_pred_safe(n,k):
    if _has('mass_pred'):
        return mass_pred(n,k)
    BASE = PHI**PHI; ME = 0.510999
    return ME * (2.0**n) * (BASE**k)

def df_en_ready():
    # согласованный df из 4-й/5-й или 2-й ячейки
    if _has('df'):
        return df.copy()
    if _has('df_en'):
        return df_en.copy()
    if _has('build_df_en_suite'):
        dfe, _ = build_df_en_suite()
        return dfe
    raise RuntimeError("Нужна хотя бы 2-я ячейка (build_df_en_suite).")

# --- ключевые формы δ13 и ρ8 под 13/8 ---
def delta13_of(n, k, c, sign_q, p=1, q=1, r=2, s=DELTA_S_STAR):
    return (p*n + q*k + r*c + s*sign_q) % 13
def rho8_of(n, k, c, sign_q, u=2, v=1, w=1, x=RHO_X_STAR):
    return (u*n + v*k + w*c + x*sign_q) % 8

# ---------- набор базовых данных ----------
df0 = df_en_ready()
# пересчёт δ/ρ на всякий случай по (s*, x*)
p,q,r = 1,1,2; u,v,w = 2,1,1
df0['delta13'] = (p*df0['n'] + q*df0['k'] + r*df0['c'] + DELTA_S_STAR*np.sign(df0['charge'])) % 13
df0['rho8']    = (u*df0['n'] + v*df0['k'] + w*df0['c'] + RHO_X_STAR*np.sign(df0['charge'])) % 8

d_e, r_e = int(df0.loc[df0['name']=='electron','delta13'].iloc[0]), int(df0.loc[df0['name']=='electron','rho8'].iloc[0])
d_n, r_n = int(df0.loc[df0['name']=='neutron','delta13'].iloc[0]),  int(df0.loc[df0['name']=='neutron','rho8'].iloc[0])

# ---------- локальные зоны для скана ----------
NEIGHBORS = {
    "(2,8) core": (2, 8, 2),   # (n0,k0,радиус Манхэттена)
    "(4,6) bridge": (4, 6, 2),
    "(10,0) N-key": (10, 0, 2),
}
# можно добавить свои: NEIGHBORS["(x,y) label"] = (x,y,R)

# фильтры
ONLY_N_GE_0_BARYONS = True
BARYON_K_EVEN = True

# окна масс (центр, полуширина)
MASS_WINDOWS = {
    "pion~140":    (140.0, 40.0),
    "kaon~500":    (495.0, 90.0),
    "nucleon~940": (938.0, 120.0),
    "J/psi~3097":  (3097.0, 350.0),
}

# веса скоринга
W_K        = 0.45   # минимальность K
W_WINDOW   = 0.40   # попадание/близость к окнам масс
W_CENTER   = 0.15   # близость к центру локальной зоны
PENALTY_NEG_N_BARYON = 0.7
PENALTY_ODD_K_BARYON = 0.7

def window_score(m_mev):
    if not MASS_WINDOWS: return 0.0
    best = 0.0
    for _, (c, half) in MASS_WINDOWS.items():
        d = abs(m_mev - c)
        s = max(0.0, 1.0 - d/half)  # внутри окна -> [0..1]
        best = max(best, s)
    return best

def local_score(n,k,m_mev,nc,kc):
    K = abs(n)+abs(k)
    sK = 1.0/(1.0 + K)
    sW = window_score(m_mev)
    dC = abs(n-nc) + abs(k-kc)
    sC = 1.0/(1.0 + dC)
    return W_K*sK + W_WINDOW*sW + W_CENTER*sC

# корзина-поисковик по локальной зоне
def scan_bucket(zone_label, n0, k0, R,
                target_delta, target_rho, c_guess, sign_q,
                is_baryon=False, top=20):
    rows=[]
    # соберём «пустые» точки решётки в данной окрестности
    occupied = set(zip(df0['n'], df0['k']))
    for n in range(n0-R, n0+R+1):
        for k in range(max(0, k0-R), k0+R+1):  # k>=0
            if (n,k) in occupied:
                continue
            # жёсткие фильтры для барионов
            if is_baryon and ONLY_N_GE_0_BARYONS and n < 0:
                continue
            if is_baryon and BARYON_K_EVEN and (k % 2 != 0):
                continue
            # ключи
            d = delta13_of(n,k,c=c_guess, sign_q=sign_q)
            r8= rho8_of(n,k,c=c_guess, sign_q=sign_q)
            if d==target_delta and r8==target_rho:
                mp = mass_pred_safe(n,k)
                sc = local_score(n,k,mp,n0,k0)
                # штрафы барионов
                if is_baryon and n < 0:
                    sc *= PENALTY_NEG_N_BARYON
                if is_baryon and (k % 2 != 0):
                    sc *= PENALTY_ODD_K_BARYON
                rows.append((zone_label, sc, abs(n)+abs(k), n, k, int(d), int(r8), mp))
    cdf = pd.DataFrame(rows, columns=['zone','score','K','n','k','delta13','rho8','mass_pred_MeV']) \
           .sort_values(['score','K'], ascending=[False,True]).head(top)
    return cdf

# --- запуск: 4 корзины в каждой зоне ---
tables = []
for label,(n0,k0,R) in NEIGHBORS.items():
    # N-key (baryon-like)
    tables.append( scan_bucket(label, n0,k0,R, d_n, r_n, c_guess=2, sign_q=0,  is_baryon=True,  top=30) )
    # E-key (lepton-like)
    tables.append( scan_bucket(label, n0,k0,R, d_e, r_e, c_guess=1, sign_q=-1, is_baryon=False, top=30) )
    # мосты
    tables.append( scan_bucket(label, n0,k0,R, d_n, r_e, c_guess=2, sign_q=0,  is_baryon=True,  top=30) )
    tables.append( scan_bucket(label, n0,k0,R, d_e, r_n, c_guess=1, sign_q=-1, is_baryon=False, top=30) )
res = pd.concat([t for t in tables if len(t)], ignore_index=True) if any(len(t)>0 for t in tables) else pd.DataFrame(columns=['zone','score','K','n','k','delta13','rho8','mass_pred_MeV'])

# ---- сводка по зонам ----
pd.set_option("display.float_format", lambda x: f"{x:,.6g}")
print("ТОП кандидатов по зонам (score↘, затем K↗):")
if len(res):
    for label in NEIGHBORS.keys():
        tbl = res[res['zone']==label]
        if len(tbl):
            print(f"\n[{label}]")
            print(tbl.head(12).to_string(index=False))
else:
    print("— кандидатов в текущих окрестностях не найдено (проверь фильтры/радиусы).")

# ---- визуализация: карта (n,k) по всем зонам с пузырями score ----
plt.rcParams['figure.figsize'] = (18,6)
fig, axs = plt.subplots(1,3)

# A) карта (n,k)
fam_colors = {'lepton':'#FF6B6B','quark':'#4ECDC4','baryon':'#45B7D1','meson':'#FFA07A','boson':'#98D8C8'}
ax = axs[0]
for f in df0['family'].unique():
    dff = df0[df0['family']==f]
    ax.scatter(dff['n'], dff['k'], s=110, alpha=0.55, edgecolors='black', linewidth=1.0,
               c=fam_colors.get(f,'#999'), label=f)
# пузырь по score
if len(res):
    sizes = (res['score']*900).clip(60, 420)
    ax.scatter(res['n'], res['k'], s=sizes, c='gold', marker='*', edgecolors='black', linewidth=1.4, alpha=0.9, label='candidates')
# маркеры e/n
e = df0[df0['name']=='electron'].iloc[0]; nrow = df0[df0['name']=='neutron'].iloc[0]
ax.scatter([e['n']],[e['k']], s=260, marker='s', c='yellow', edgecolors='black', linewidth=2, label='electron')
ax.scatter([nrow['n']],[nrow['k']], s=260, marker='D', c='lime',   edgecolors='black', linewidth=2, label='neutron')
# рамки зон
for label,(n0,k0,R) in NEIGHBORS.items():
    ax.add_patch(plt.Rectangle((n0-R-0.5, max(0,k0-R)-0.5), 2*R+1, 2*R+1,
                               fill=False, linestyle='--', linewidth=1.2, edgecolor='purple'))
ax.set_title('Тонкий 13/8-поиск: (n,k)'); ax.set_xlabel('n'); ax.set_ylabel('k'); ax.grid(True, alpha=0.3); ax.legend(fontsize=8)

# B) попадания в окна масс
ax = axs[1]
labels, counts = [], []
for name,(c,half) in MASS_WINDOWS.items():
    cnt = 0
    if len(res):
        m = res['mass_pred_MeV'].values
        cnt = int(np.sum(np.abs(m - c) <= half))
    labels.append(name); counts.append(cnt)
ax.bar(labels, counts, edgecolor='black')
ax.set_title('Попадания в окна масс'); ax.grid(True, axis='y', alpha=0.3)

# C) распределение K
ax = axs[2]
if len(res):
    ax.hist(res['K'], bins=range(int(res['K'].min()), int(res['K'].max())+2), edgecolor='black', alpha=0.85)
    ax.set_title('Распределение K (локальный поиск)'); ax.set_xlabel('K'); ax.set_ylabel('кол-во'); ax.grid(True, axis='y', alpha=0.3)
else:
    ax.axis('off'); ax.text(0.5,0.5,"кандидатов нет", ha='center', va='center')

plt.tight_layout(); plt.show()